# Assignment 2: Iterative-Deepening Search

Prashant Kumar Thakur

## Overview

I have implemented the iterative-deepening search algorithm as discussed in our Week 2 lecture notes and as shown in figures 3.17 and 3.18 in our text book. It has been applied to the 8-puzzle and a second puzzle "Water-Jug Problem".

## Required Code

In this jupyter notebook, implement the following functions:

  * `iterativeDeepeningSearch(startState, actionsF, takeActionF, goalState, goalTestF, maxDepth)`
  * `depthLimitedSearch(startState, actionsF, takeActionF, goalState, goalTestF, depthLimit)`
  
`depthLimitedSearch` is called by `iterativeDeepeningSearch` with `depthLimit`s of $0, 1, \ldots, $ `maxDepth`. Both must return either the solution path as a list of states, or the strings `cutoff` or `failure`.  `failure` signifies that all states were searched and the goal was not found. 

Each receives the arguments

  * the starting state, 
  * a function `actionsF` that is given a state and returns a list of valid actions from that state,
  * a function `takeActionF` that is given a state and an action and returns the new state that results from applying the action to the state,
  * the goal state,
  * a function `goalTestF` that is given a state and the goal state and returns `True` if the state satisfies the goal, and
  * either a `depthLimit` for `depthLimitedSearch`, or `maxDepth` for `iterativeDeepeningSearch`.

Use your solution to solve the 8-puzzle.
Implement the state of the puzzle as a list of integers. 0 represents the empty position. 

Required functions for the 8-puzzle are the following.

* `findBlank_8p(state)`: return the row and column index for the location of the blank (the 0 value).,
* `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in state. Return them in the order left, right, up, down, though only if each one is a valid action.,
* `takeActionF_8p(state, action)`: return the state that results from applying action in state.,
* `printPath_8p(startState, goalState, path)`: print a solution path in a readable form.

The functions depthLimitedSearch is implemented which takes five arguments. Initially the startState is matched to the goalState to ensure if the result has been achieved. The function provided for the assignment were not modified much. The only modification was to append the states that the search transverses to achieve the path. The depthLimitedSearch returns the empty path when the startState and goalState matches to remove duplicates in the search path when we do a iterative deepening search. If there are multiple solution to a state, the depth limited search tends to find the one that it encounters which is not necessarily the optimal path as it explores the deepest location of one child and then moves to another. Because of which if it find a solution path at level-2 (say) it won't bother to check if there exists a solution at level-1 at another node. This has been clarified at the bottom of the notebook with an explanation. So, depth limited search helps to remove the infinite loop problem thought it is not the best method to used based on the nature of the graph.

* `"generateNpArray"` function is implemented to change the one-dimentional array into 2 dimentional array and viceversa. This approach is done to make change to this function to create facilitate different length puzzle. Like- if we need to have 4X4 puzzle then all we need to do is switch the parameters in this function to generate 4X4 2-d array and 16 element in 1-d array. The parameter it takes is state and list boolean. By default the list boolean is fixed to False so that the state passed is changed from 1-d to 2-d array. When we have to return the result state, the 2-d array is converted to 1-d and for that particular conversion list=True is used. For instance, the function "takeActionF_8p" initially computes the 2-d array by passing only state (1-d array) however at the end of the computation, it returns 1-d array and the conversion from 2-d array is done using "generaateNpArray" function by passing list=True as second parameter.

* `"printState_8p"` is another function used to print the states in a matrix format. The function first generates the 2-d array using the numpy module and then the values in the array is iterated to get each row and a string is computed by removing "[","]" from the list and replacing a blank space for "0". For instance,
If we input [1, 0, 3, 4, 2, 5, 6, 7, 8] as the state to this function, it first converst this array into the following array and then a required format is computed dumping brackets and replacing 0 with a blank space.
    
    Internal conversion:
       [[1 0 3]
         [4 2 5]
         [6 7 8]]
    Output:
        1   3
        4 2 5
        6 7 8

* `findBlank_8p` function is used to find the position of 0 in the array. This function basically uses the numpy function to get the index of the element which matches the argument passed. The method `argwhere` is used to find the index which returns a list of matching position. Finally, the index value is returned as a tuple. So for a given state [1, 0, 3, 4, 2, 5, 6, 7, 8], it is first converted into 2-d array and then the index is computed and the value is returned which in this case is (0, 1).

* `actionsF_8p` function is used to generate the list of action that could be applied to a given state. For instance, if the blank is present in the array as shown below then the blank can be moved towards right, left, down.Similarly, if the blank was present in the middle of the array i.e. in place of 2 (example shown above) then the state could have 4 successors with blank that can move left, right, up, down. This function checks for the position of the blank (or zero) in the array and compute the strategies about where it can move. Based on the allowed movements the list is prepared and returned from the function. So if we consider the above example then the list returned is ["left", "right", "down"] as these are the only allowed states.

        1   3
        4 2 5
        6 7 8

* `takeActionF_8p` function simply computes the child state provided the state and the action applied to the state. The 1-d array is first changed into 2-d array so that the swapping of the element becomes easy. Finally, based on the action the 2-d state is computed after swapping the values. The final result returned from the function is the 1-d array for which the function `generateNpArray` is used by passing list=True.


In [51]:
import numpy as np
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            #Add startState to front of solution path, in result, returned by depthLimitedSearch
            result.insert(0,startState)
            return result
    return 'cutoff'


def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    if state == goalState:
        # Return the goal path is already returned from the result so return empty list to remove duplication
        return []

    if depthLimit is 0:
        # signal that the depth limit was reached
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        childState = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result is 'cutoff':
            cutoffOccurred = True
        elif result is not 'failure':
            #Add childState to front of partial solution path, in result, returned by depthLimitedSearch
            result.insert(0,childState)
            return result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'


def generateNpArray(state,list=False):
    #Create a general function that can be used to format the arrays for input and output.
    # Return the array in 1-D with 9 elements
    if list:
        return state.reshape(1, 9)[0].tolist()
    # Return the array in 2-D with 3 rows, 3 column
    else:
        return np.array(state).reshape(3, 3)


def printState_8p(state):
    np_array = generateNpArray(state)
    print('\n'.join(str(a).strip('[]').replace('0', ' ') for a in np_array))


def findBlank_8p(state):
    np_array = generateNpArray(state)
    idx = np.argwhere(np_array == 0)
    return tuple(idx[0])


def actionsF_8p(state):
    position = findBlank_8p(state)
    action = ['left', 'right', 'up', 'down']
    if position[0] == 0:
        action.remove('up')
    if position[0] == 2:
        action.remove('down')
    if position[1] == 0:
        action.remove('left')
    if position[1] == 2:
        action.remove('right')
    return action


def takeActionF_8p(state, action):
    np_array = generateNpArray(state)
    position = findBlank_8p(state)
    try:
        if action == 'left':
            np_array[position], np_array[(position[0], position[1]-1)] = np_array[(position[0],position[1]-1)], np_array[position]
        if action == 'right':
            np_array[position], np_array[(position[0], position[1]+1)] = np_array[(position[0],position[1]+1)], np_array[position]
        # swap the value with the top row whose row is 1 unit less.
        if action == 'up':
            np_array[position], np_array[(position[0]-1, position[1])] = np_array[(position[0]-1,position[1])], np_array[position]

        if action == 'down':
            np_array[position], np_array[(position[0]+1, position[1])] = np_array[(position[0]+1,position[1])], np_array[position]
    except IndexError:
        print("Wrong Action applied.\n Action ={} on .. \n{}".format(action,printState_8p(state)))
    return generateNpArray(np_array,list=True)


def printPath_8p(startState, goalState, path):
    print("Path from\n{} \n to {} \n is {} nodes long.\n".format(startState,goalState,len(path)))
    for item in path:
        print("\n")
        printState_8p(item)


Here are some example results.

In [52]:
startState = [1, 0, 3, 4, 2, 5, 6, 7, 8]

In [53]:
printState_8p(startState)  

1   3
4 2 5
6 7 8


In [54]:
findBlank_8p(startState)

(0, 1)

In [55]:
actionsF_8p(startState)

['left', 'right', 'down']

In [56]:
takeActionF_8p(startState, 'down')

[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [57]:
printState_8p(takeActionF_8p(startState, 'down'))

1 2 3
4   5
6 7 8


In [58]:
goalState = takeActionF_8p(startState, 'down')

In [59]:
newState = takeActionF_8p(startState, 'down')

In [60]:
newState == goalState

True

In [61]:
startState

[1, 0, 3, 4, 2, 5, 6, 7, 8]

In [62]:
path = depthLimitedSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [63]:
path = depthLimitedSearch(startState, goalState, actionsF_8p, takeActionF_8p, 5)
path

[[0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Notice that `depthLimitedSearch` result is missing the start state.  This is inserted by `iterativeDeepeningSearch`.

But, when we try `iterativeDeepeningSearch` to do the same search, it finds a shorter path!

In [64]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]]

# Description of DLS and IDLS behavior.

We can see that when we do an iterative deepening search with a start state [1, 0, 3, 4, 2, 5, 6, 7, 8] to achieve a goal state [1, 2, 3, 4, 0, 5, 6, 7, 8], we reach the goal state with an optimal solution. But this is not true for the depth limited search. The DLS seems to find the longest path based on the depthlimite passed to it. In the above example, the depth limit was passed to be 3 so the DLS tries to find all the child to the leftmost node and try to find if it gets to a solution. Since, the action are strictly arranged as ["left","right","up","down"] the DLS first goes through all the leftmost child and finally gets a solution at the third level with different path. In order to clarify on this theme, lets change the depth limit to the function. For instance, we change the depth limit to 3 and the response we get is [[0, 1, 3, 4, 2, 5, 6, 7, 8],[1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]].  Again with the same startState and goalState with depth limit of 5, we get a different path as a solution.  [[0, 1, 3, 4, 2, 5, 6, 7, 8],[1, 0, 3, 4, 2, 5, 6, 7, 8], [0, 1, 3, 4, 2, 5, 6, 7, 8], [1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]].

This clearly indicates that the depth limited search is trying to find the deepest goal state which could be found at the leftmost part of the graph. 

However, the iterative deepening search tries to find the optimal solution because we gradually increase the depth of the graph and see if we can achieve the goal state at the given depth level. Since, there exists a path at level 1 which is matched first regardless of some higher maximum depth allowed. Because of this nature, every node at a given level is checked first and then the lower nodes are explored. Therefore, it finds the optimal path to a given goal state where depth limited search couldn't reach.

To get the similar result with the DLS, we have to try to use the minimum depth limit. For the above example if we set the depth limit to 2, then we get the same path as found by iterative deepening search. Hence, iterative deepening search is better over depth limited search. 

In [65]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

'cutoff'

In [66]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 5)
path

'cutoff'

Humm...maybe we can't reach the goal state from this state.  We need a way to randomly generate a valid start state.

In [72]:
import random

In [74]:
random.choice(['left', 'right'])

'right'

In [75]:
def randomStartState(goalState, actionsF, takeActionF, nSteps):
    state = goalState
    for i in range(nSteps):
        state = takeActionF(state, random.choice(actionsF(state)))
    return state

In [76]:
startState = randomStartState(goalState, actionsF_8p, takeActionF_8p, 10)
startState

[4, 1, 3, 2, 5, 8, 6, 7, 0]

In [77]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 20)
path

[[4, 1, 3, 2, 5, 8, 6, 7, 0],
 [4, 1, 3, 2, 5, 0, 6, 7, 8],
 [4, 1, 3, 2, 0, 5, 6, 7, 8],
 [4, 1, 3, 0, 2, 5, 6, 7, 8],
 [0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Let's print out the state sequence in a readable form.

In [78]:
for p in path:
    printState_8p(p)
    print()

4 1 3
2 5 8
6 7  

4 1 3
2 5  
6 7 8

4 1 3
2   5
6 7 8

4 1 3
  2 5
6 7 8

  1 3
4 2 5
6 7 8

1   3
4 2 5
6 7 8

1 2 3
4   5
6 7 8



Here is one way to format the search problem and solution in a readable form.

In [79]:
printPath_8p(startState, goalState, path)

Path from
[4, 1, 3, 2, 5, 8, 6, 7, 0] 
 to [1, 2, 3, 4, 0, 5, 6, 7, 8] 
 is 7 nodes long.



4 1 3
2 5 8
6 7  


4 1 3
2 5  
6 7 8


4 1 3
2   5
6 7 8


4 1 3
  2 5
6 7 8


  1 3
4 2 5
6 7 8


1   3
4 2 5
6 7 8


1 2 3
4   5
6 7 8


Download [A2grader.tar](A2grader.tar) and extract A2grader.py from it.

In [80]:
%run -i A2grader.py


Searching this graph:
 {'a': ['b', 'z', 'd'], 'b': ['a'], 'e': ['z'], 'd': ['y'], 'y': ['z']}

Looking for path from a to y with max depth of 1.
 5/ 5 points. Your search correctly returned cutoff

Looking for path from a to y with max depth of 5.
10/10 points. Your search correctly returned ['a', 'z']

Testing findBlank_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
 5/ 5 points. Your findBlank_8p correctly returned 2 1

Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
10/10 points. Your actionsF_8p correctly returned ['left', 'right', 'up']

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8],up)
10/10 points. Your takeActionsF_8p correctly returned [1, 2, 3, 4, 0, 6, 7, 5, 8]

Testing iterativeDeepeningSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],[0, 2, 3, 1, 4,  6, 7, 5, 8], actionsF_8p, takeActionF_8p, 5)
20/20 points. Your search correctly returned [[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]]

Testing iterativeDeepeningSearch([5, 2, 

# Extra Example

For an extra example for this assignment, I have tried to solve a Water-jug problem. 

### Summary of Example
There are 2 jugs which are 4-gallon and 3-gallon with no mesurement marking on them. You can fill as much water as you can and pour the water to empty the vessel as per your requirement. The ultimate goal is to find the path such that 4-gallon jug have exactly 2 gallon of water in it at the end.

The functional representation of the function is provided below. The function "action_waterjug" does the necessary calculation to find the next states that are possible from a given state. The function "takeAction_waterjug" returns the action that it receives. So there is not much that it does but to make the test case consistent with the IDLS function. The "action_waterjug" takes the state as input and tries to find all the states that could be possible for the case. The state are represented as (x,y) in tuple where x denotes the amount of water in 4-gallon jug and y denotes the amount of water in 3-gallon jug. So the valid input would be constructed with x<=4 and y<=3 with no fractional values, however, the IDLS would try to find the solution for the given state and match it to the goal state. In order to describe the task done in the function "action_waterjug", I would give an example how the states are computed.

Let's suppose the initial state is (4,0) so either 4-gallon jug remains filled and 3-gallon jug can be filled up to produce(4,3) or both jugs could be emptied to produce (0,0) or 3-gallon jug can be filled with the water from 4-gallon jug to give (1,3). So the child state of (4,0) could be (4,3),(0,0),(1,3).

                         (4,0)
                         / | \ 
                        /  |  \
                   (4,3) (1,3) (0,0)
 
 Now the IDLS is used to find the path to get to the goal state and the actual path is printed.

In [81]:
def action_waterjug(state):
    result = set()
    x,y = state[0],state[1]
    if x < 4:
        result.add((4, y))
    if y < 3:
        result.add((x, 3))
    if x > 0:
        result.add((0, y))
    if y > 0:
        result.add((x, 0))
    if y > 0 and ((x+y <= 4) and (x+y > 0)):
        if y-(4-x) >=0:
            result.add((4, y-(4-x)))
    if x > 0 and ((x+y >= 3) and (x+y > 0)):
        result.add((x-(3-y), 3))
    if (x+y) <= 4 and y > 0:
        val = ((x+y),0)
        result.add(val)
    if (x+y) <= 3 and x > 0:
        val =(0,x+y)
        result.add(val)
    return list(result)

def takeAction_waterjug(state,action):
    return action

In [82]:
startState = (4,0)
goalState = (2,0)
action_waterjug(startState)

[(1, 3), (0, 0), (4, 3)]

In [83]:
iterativeDeepeningSearch(startState,goalState,action_waterjug,takeAction_waterjug,3)

'cutoff'

In [84]:
iterativeDeepeningSearch(startState,goalState,action_waterjug,takeAction_waterjug,8)

[(4, 0), (1, 3), (1, 0), (0, 1), (4, 1), (2, 3), (2, 0)]